# pyESM WORKFLOW

CASE A 

Generation of a new model from scratch, including database and data input

(settings -> use_existing_database: False)

In [ ]:
# Create an instance of Model class based on the model settings.
# This generates the model's set input file/s to be filled 
# and the blank SQL database.
from src.frontend.model import Model

test = Model(
    log_level='debug',
    log_format='minimal',
)

In [ ]:
# load model sets from the xlsx files filled by the user
# Note: data validation in excel according to the constant file is envisaged
test.load_model_coordinates()

In [ ]:
# Once sets has compiled in excel and loaded in the database, 
# blank database is generated.
# Note: an error is raised at the first launch of the method, 
# disappearing the second run.
test.generate_blank_database()

In [ ]:
# Load input file/s data into the sql database.
# Can be used to update data in the sql database without generating a new 
# Model instance
test.load_data_files_to_database()

In [ ]:
# Initialize cvxpy problem.and
# Fetch data from sql database to cvxpy exogenous variables,
# (to be done: generate equations and objective function)
test.initialize_problem()

CASE B

Generation of a new model working with existing database and data input files.

(settings -> use_existing_database: True)

In [1]:
# Generating a Model instance referring to existing model folder and input data.
from src.frontend.model import Model

test = Model(
    log_level='debug',
    log_format='minimal',
)

test.load_model_coordinates()
test.initialize_problem()

INFO | Model | 'Model' object initialization...
INFO | Model.file_manager | 'FileManager' object generated.
INFO | Model | 'Model' object: loading settings.
DEBUG | Model.file_manager | File 'settings.yml' loaded.
INFO | Model | 'Model' object: loading paths from settings.
INFO | Model.core | 'Core' object initialization...
INFO | Model.core | Skipping model directory generation.
INFO | Model.core.sql_manager | 'SQLManager' object Generated.
INFO | Model.core.index | 'Index' object initialization...
DEBUG | Model.core.index | Loading variables table headers and label headers to Index.
INFO | Model.core.index | 'Index' object initialized.
INFO | Model.core.database | 'Database' object initialization...
INFO | Model.core.database | 'Database' object initialized.
INFO | Model.core.problem | 'Problem' object initialization...
INFO | Model.core.problem | 'Problem' object initialized.
INFO | Model.core | 'Core' initialized.
INFO | Model.pbi_manager | 'PBIManager' object generated.
INFO | Mod

In [2]:
# In case it is needed, update data without re-generate a new instance of Model class.
test.update_database_and_problem()

DEBUG | Model.core.sql_manager | Connection to 'database.db' opened.
INFO | Model.core.database | Loading data input file/s filled by the user to SQLite database.
DEBUG | Model.file_manager | Excel file 'input_data.xlsx' loaded.
DEBUG | Model.core.sql_manager | SQLite table 'v' - 18 entries deleted.
DEBUG | Model.core.sql_manager | SQLite table 'v' - 18 entries added.
DEBUG | Model.core.sql_manager | SQLite table 'u' - 12 entries deleted.
DEBUG | Model.core.sql_manager | SQLite table 'u' - 12 entries added.
DEBUG | Model.core.sql_manager | SQLite table 'Y' - 4 entries deleted.
DEBUG | Model.core.sql_manager | SQLite table 'Y' - 4 entries added.
DEBUG | Model.core.sql_manager | Connection to 'database.db' closed.
INFO | Model.core | Initialize variables dataframes (cvxpy objects, filters dictionaries).
DEBUG | Model.core.problem | Generating variable 'v' dataframe (cvxpy object, filter dictionary).
DEBUG | Model.core.problem | Generating variable 'u' dataframe (cvxpy object, filter dict

TESTS ---

In [ ]:
test.core.index.sets.technologies

In [ ]:
variable = test.core.index.variables.X
variable.data['variable'][0]

In [ ]:
variable=test.core.index.variables.u

test.core.database.sqltools.open_connection()
raw_data = test.core.database.sqltools.filtered_table_to_dataframe(
    table_name=variable.symbol,
    filters_dict=variable.data['filter'][0]
)
test.core.database.sqltools.close_connection()

raw_data

In [ ]:
from src.util import constants

index_label = variable.table_headers[variable.shape[0]][0]
columns_label = variable.table_headers[variable.shape[1]][0]

index_items = variable.data['filter'][0][index_label]
columns_items = variable.data['filter'][0][columns_label]

pivoted_data = raw_data.pivot_table(
    index=index_label, 
    columns=columns_label, 
    values=constants._STD_VALUES_FIELD['values'][0],
    aggfunc='first',
).reindex(index=index_items, columns=columns_items)

pivoted_data

In [ ]:
test.core.index.variables.Y.data['filter'][0]

In [ ]:
test.core.database.sqltools.open_connection()

var = test.core.database.sqltools.filtered_table_to_dataframe(
    table_name='u',
    filters_dict={
        's_Name': ['Reference'],
        'dt_Name': [2023],
        'f_Name': ['Energy', 'Steel'],
        't_Name': ['PV power plant', 'Steel factory', 'Gas power plant']
    }
)

# filter_1 = test.core.database.sqltools.get_related_table_keys(
#     child_column_name='f_Name',
#     parent_table_name='_set_FLOWS',
#     parent_table_fields={
#         'f_Category': ['Product flow'],
#     }
# )

# filter_2 = test.core.database.sqltools.get_related_table_keys(
#     child_column_name='dt_Name',
#     parent_table_name='_set_DATETIME',
#     parent_table_fields={
#         'dt_Name': [2023],
#     }
# )

# var_custom_filter = test.core.database.sqltools.filtered_table_to_dataframe(
#     table_name='v',
#     filters_dict={
#         **filter_1,
#         **filter_2,
#         **{'f_Name': ['Steel']},
#     }
# )

test.core.database.sqltools.close_connection()

# filter_1, filter_2
var

In [ ]:
var.pivot(index='f_Name', columns='t_Name', values='values')

In [ ]:
# delete all the information included in the model data folder, if necessary
test.core.model_dir_cleanup()